In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import tarfile
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split

from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor

import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data.dataloader import DataLoader

In [2]:
#the path of the data
data_dir = './input/gender-classification-dataset'

['archive.zip', 'Training', 'Validation']
['archive.zip', 'Training', 'Validation']


In [7]:
#resizing the input images
transform = transforms.Compose([transforms.Resize((100, 100)), transforms.ToTensor()])

In [8]:
# read the training and testing data sets
dataset_train = ImageFolder(data_dir+"/Training", transform = transform)
dataset_val = ImageFolder(data_dir+"/Validation", transform = transform)

In [15]:
# creating a validatio set from the training data set
train_size = 45000
val_size = len(dataset_train) - train_size

data_train, data_val = random_split(dataset_train, [train_size, val_size])
len(data_train), len(data_val)

(45000, 2009)

In [16]:
# declearing the batch size
batch_size = 128

In [17]:
# prepairing the data loaders
train_dl = DataLoader(data_train, batch_size, shuffle = True, num_workers= 4, pin_memory=True)
val_dl = DataLoader(data_val, batch_size*2, num_workers=4, pin_memory=True)
test_dl = DataLoader(dataset_val, batch_size*2, num_workers=4, pin_memory=True)

In [23]:

# the needed fuctions to train the model
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))
        
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [24]:
# the original model structure
class GenCnnModel(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3,stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Output size: 64 x 50 x 50

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3,stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Output size: 128 x 25 x 25

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 256, kernel_size=3,stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Output size: 256 x 12 x 12

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3,stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Output size: 512 x 6 x 6

            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3,stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Output size: 512 x 3 x 3

            nn.Flatten(),
            nn.Linear(512*3*3, 1024),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Linear(256, 32),
            nn.ReLU(),
            nn.Linear(32, 2))
    def forward(self, xb):
        return self.network(xb)

In [25]:
# creating the original model
model = GenCnnModel()
model

GenCnnModel(
  (network): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (16): ReLU()
    (17): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")
device

device(type='cuda')

In [28]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [29]:
#transform al the data loader to the device.
train_loader = DeviceDataLoader(train_dl, device)
val_loader = DeviceDataLoader(val_dl, device)
test_loader = DeviceDataLoader(test_dl, device)
to_device(model, device);

In [30]:
#the evaluation and training functions.
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [31]:
#setting the model into the device settings.
model = to_device(GenCnnModel(), device)

In [32]:
# initial evaluation
evaluate(model, val_loader)

{'val_loss': 0.6932201385498047, 'val_acc': 0.5093786120414734}

In [33]:
num_epochs = 10 
opt_func = torch.optim.Adam
lr = 0.001

In [34]:
import time

In [37]:
# training the  odel
start =int(round(time.time()*1000))
history = fit(num_epochs, lr, model, train_loader, val_loader, opt_func)
# end =int(round(time.time()*1000))
# torch.cuda.synchronize()
time_elapsed = int(round(time.time()*1000)) -start
# print(start.elapsed_time(end))
print ('test time elapsed {}ms'.format(time_elapsed))
print ('test time elapsed {}s'.format(time_elapsed/1000))

Epoch [0], train_loss: 0.6944, val_loss: 0.6938, val_acc: 0.4864
Epoch [1], train_loss: 0.6931, val_loss: 0.6934, val_acc: 0.4864
Epoch [2], train_loss: 0.6931, val_loss: 0.6935, val_acc: 0.4864
Epoch [3], train_loss: 0.6931, val_loss: 0.6932, val_acc: 0.4864
Epoch [4], train_loss: 0.6931, val_loss: 0.6939, val_acc: 0.4864
Epoch [5], train_loss: 0.6931, val_loss: 0.6939, val_acc: 0.4864
Epoch [6], train_loss: 0.6931, val_loss: 0.6937, val_acc: 0.4864
Epoch [7], train_loss: 0.6931, val_loss: 0.6938, val_acc: 0.4864
Epoch [8], train_loss: 0.6931, val_loss: 0.6936, val_acc: 0.4864
Epoch [9], train_loss: 0.6931, val_loss: 0.6935, val_acc: 0.4864
test time elapsed 2223936ms
test time elapsed 2223.936s


In [39]:
evaluate(model, val_loader)
start =int(round(time.time()*1000))
evaluate(model, val_loader)
end =int(round(time.time()*1000))
# torch.cuda.synchronize()
time_elapsed = int(round(time.time()*1000)) -start
# print(start.elapsed_time(end))
print ('test time elapsed {}ms'.format(time_elapsed))
print ('test time elapsed {}s'.format(time_elapsed/1000))

test time elapsed 6612ms
test time elapsed 6.612s


In [37]:
evaluate(model, test_loader)

{'val_loss': 0.11760838329792023, 'val_acc': 0.9614548683166504}